In [2]:
# imports
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
# descriptive statistics
df = pd.read_csv('Data/monthly/data.csv')

df_ds = df.describe()
df_ds = df_ds.transpose()
df_ds

,count,mean,std,min,25%,50%,75%,max
Absatzzahlen,60.0,1.671455e+04,2894.585199,1.034600e+04,1.453700e+04,1.644050e+04,1.925550e+04,2.293700e+04
BMW Finanzierung,60.0,5.949000e+01,9.782182,4.240000e+01,5.193750e+01,5.892500e+01,6.610000e+01,8.400000e+01
BMW Leasing,60.0,6.279250e+01,11.141748,4.125000e+01,5.395000e+01,6.200000e+01,7.176250e+01,8.600000e+01
Auto Finanzierung,60.0,6.728833e+01,12.040329,4.000000e+01,5.962500e+01,6.550000e+01,7.670000e+01,9.375000e+01
Auto Leasing,60.0,4.812000e+01,8.392653,2.720000e+01,4.525000e+01,4.735000e+01,5.262500e+01,7.600000e+01
Gebrauchtwagenindex,60.0,1.418933e+02,15.463855,1.147000e+02,1.297500e+02,1.401500e+02,1.510000e+02,1.716000e+02
EURIBOR 3M,60.0,1.273870e+00,1.930329,-5.820000e-01,-5.235500e-01,-1.013000e-01,3.459250e+00,3.971600e+00
Geschäftsklima,60.0,9.050833e+01,5.249448,7.500000e+01,8.660000e+01,8.965000e+01,9.320000e+01,1.010000e+02
Geschäftslage,60.0,9.224667e+01,5.730485,7.860000e+01,8.827500e+01,9.310000e+01,9.722500e+01,1.017000e+02
Geschäftserwartungen,60.0,8.895500e+01,6.799263,7.100000e+01,8.435000e+01,8.740000e+01,9.460000e+01,1.022000e+02


In [ ]:
# seasonal adjustment

from statsmodels.tsa.seasonal import STL

# List of columns to be seasonally adjusted
seasonal_cols = [
  'Absatzzahlen', 'BMW Finanzierung', 'BMW Leasing', 'Auto Finanzierung', 'Auto Leasing',
  'Geschäftsklima', 'Geschäftslage', 'Geschäftserwartungen', 'Verbraucherpreisindex',
  'Arbeitslosenquote', 'Bestand Arbeitslose', 'Zugang Arbeitslose', 'Abgang Arbeitslose'
]

df_sa = df.copy()
df_sa.columns = df_sa.columns.str.strip() # Strip whitespace from column names
df_sa['DATE'] = pd.to_datetime(df_sa['DATE'])
df_sa = df_sa.set_index('DATE')

for col in seasonal_cols:
  stl = STL(df_sa[col], period=12, robust=True)
  res = stl.fit()
  df_sa[col] = res.trend + res.resid  # seasonally adjusted

df_sa = df_sa.reset_index()

df_sa

,DATE,Absatzzahlen,BMW Finanzierung,BMW Leasing,Auto Finanzierung,Auto Leasing,Gebrauchtwagenindex,EURIBOR 3M,Geschäftsklima,Geschäftslage,...,Zugang Arbeitslose,Abgang Arbeitslose,tavg,tmin,tmax,prcp,wdir,wspd,wpgt,pres
0,2020-01-31,20197.056055,67.643871,81.596561,61.358966,34.170965,118.6,-0.3911,98.267185,103.826453,...,568735.099659,524308.088261,4.561290,-2.7,13.4,49.0,229.774194,15.341935,59.3,1021.090323
1,2020-02-29,19996.253807,63.831304,79.003916,63.371881,37.399108,119.5,-0.4089,85.437142,80.246985,...,585792.895041,517012.825158,6.282759,0.8,14.4,89.2,241.896552,19.841379,77.8,1009.951724
2,2020-03-31,20037.985951,62.868894,46.441574,59.629593,34.745923,119.4,-0.4166,85.249613,95.147574,...,587399.801317,509569.133724,5.719355,-4.0,16.8,47.6,206.096774,15.345161,79.6,1017.764516
3,2020-04-30,19640.703601,66.051952,77.737069,60.229097,34.903892,116.1,-0.2541,86.375177,82.241084,...,583905.870812,498685.856264,10.700000,-2.0,23.0,7.7,202.033333,13.520000,50.0,1018.456667
4,2020-05-31,8489.754848,62.523694,79.622751,61.569152,38.912760,114.7,-0.2720,87.149813,83.296517,...,542199.566571,495945.915068,12.293548,0.6,24.1,26.9,233.838710,13.419355,46.3,1019.329032
5,2020-06-30,19631.433705,63.658155,76.838510,59.269915,39.968844,116.0,-0.3760,88.050757,84.849025,...,519170.664768,494023.972901,18.760000,9.0,29.8,53.3,168.500000,12.653333,50.0,1012.686667
6,2020-07-31,19515.601434,58.670709,76.381685,59.419881,40.761096,120.5,-0.4441,88.863557,85.917119,...,511641.028724,495479.334094,17.987097,9.8,28.3,46.7,252.322581,12.058065,59.3,1015.345161
7,2020-08-31,18811.107759,60.028149,74.502541,57.644269,39.915919,124.1,-0.4797,89.869731,87.249971,...,510236.995637,492419.706227,21.154839,11.4,34.6,37.5,180.709677,11.896774,66.7,1013.280645
8,2020-09-30,19636.144051,59.873327,74.045313,59.607731,42.043805,126.4,-0.4914,89.288746,88.691963,...,496382.109076,477452.326103,15.420000,5.9,30.3,44.8,204.166667,10.916667,50.0,1016.486667
9,2020-10-31,19115.791937,61.490814,73.356609,55.226831,42.835956,127.2,-0.5091,90.269909,89.973420,...,519446.174981,498991.402665,11.112903,3.8,21.7,70.0,191.129032,13.651613,53.7,1011.454839


In [ ]:
# 

In [ ]:
# feature engineering

In [ ]:
# modeling

In [ ]:
# plotting